In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from tqdm.notebook import tqdm

from pymorphy2 import MorphAnalyzer

In [2]:
train_df = pd.read_csv('data/vacancies_train.csv')
test_df = pd.read_csv('data/vacancies_test.csv')

In [5]:
cleaned_train = pd.read_csv('data/preprocessed_data_train.csv')
cleaned_test = pd.read_csv('data/preprocessed_data_test.csv')

In [6]:
cleaned_train

,text,target
0,водитель экспедитор категория м газель тент га...,42500.0
1,обязанность списывание материал производство п...,50000.0
2,крупный федеральный сеть магазин магнит требов...,31500.0
3,крупный федеральный сеть магазин мужской женск...,36700.0
4,обязанность уборка производственный площадь ку...,37000.0
...,...,...
16624,обязанность обеспечение общественный порядок о...,67500.0
16625,обязанность оказывать квалифицировать медицинс...,50000.0
16626,условие официальный оформление тк рф график ра...,50000.0
16627,обязанность своевременный качественный выполне...,75000.0


In [8]:
train_df['cleaned'] = cleaned_train.text
test_df['cleaned'] = cleaned_test.text

In [9]:
train_df

,name,experience,schedule,employment,description,key_skills,salary,cleaned
0,Водитель-экспедитор,От 3 до 6 лет,Полный день,Полная занятость,"Водитель-экспедитор категории В,С (а\м ГАЗЕЛЬ ...",[],42500.0,водитель экспедитор категория м газель тент га...
1,Кладовщик-комплектовщик/оператор 1С,От 1 года до 3 лет,Полный день,Полная занятость,Обязанности: Списывание материалов в произво...,"['знание пк', 'работа в 1С', 'Работа в команде...",50000.0,обязанность списывание материал производство п...
2,"Продавец магазина ""Магнит"" (Кировский р-он, В....",Нет опыта,Сменный график,Полная занятость,"В крупнейшую Федеральную сеть магазинов ""Магни...",[],31500.0,крупный федеральный сеть магазин магнит требов...
3,Администратор магазина (ТЦ Spar),От 1 года до 3 лет,Полный день,Полная занятость,Крупная федеральная сеть магазинов мужской и ж...,[],36700.0,крупный федеральный сеть магазин мужской женск...
4,Мойщик посуды (кулинария),Нет опыта,Сменный график,Полная занятость,Обязанности: Уборка производственных площадей...,"['Пользователь ПК', 'Грамотная речь', 'Работа ...",37000.0,обязанность уборка производственный площадь ку...
...,...,...,...,...,...,...,...,...
16624,Полицейский специального полка полиции,Нет опыта,Полный день,Полная занятость,Обязанности: обеспечение общественного поряд...,"['Грамотная речь', 'Обучение и развитие', 'Упр...",67500.0,обязанность обеспечение общественный порядок о...
16625,Врач стоматолог-ортопед (г. Новосибирск),Нет опыта,Полный день,Полная занятость,Обязанности: Оказывает квалифицированную меди...,['Медицинская документация'],50000.0,обязанность оказывать квалифицировать медицинс...
16626,Фармацевт,От 1 года до 3 лет,Полный день,Полная занятость,Условия: официальное оформление по ТК РФ; гра...,"['Пользователь ПК', 'Работа в команде', 'Грамо...",50000.0,условие официальный оформление тк рф график ра...
16627,Геодезист (автодороги),От 1 года до 3 лет,Вахтовый метод,Полная занятость,Обязанности: своевременное и качественное вып...,[],75000.0,обязанность своевременный качественный выполне...


# EDA

In [2]:
import fasttext
import fasttext.util

In [ ]:
fasttext.util.download_model('ru', if_exists='ignore')

In [14]:
fasttext.load_model('cc.ru.300.bin')

ValueError: fastText: trained model cannot be opened!

In [15]:
fasttext.